<a href="https://colab.research.google.com/github/kalinosia/freeCodeCamp_Machine_Learning_with_Python/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

## Imports

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

## Get data

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-02-15 08:16:05--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  60.2MB/s    in 0.4s    

2022-02-15 08:16:06 (60.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

### review/see data

In [5]:
print( df_ratings.values[0])

[276725 '034545104X' 0.0]


In [6]:
# add your code here - consider creating a new cell for each section of code##############################################################################################################################################
##########################################################################################################################################################################################################################
#print(df_books.head(5))
#print(df_books['title'])
#print(df_ratings.loc[(df_ratings["isbn"]=="0061054143")])

#print(len(df_ratings['user']))
print(df_ratings.shape)
#print( df_ratings.values)
#print(df_ratings.to_numpy())


(1149780, 3)


```
BX-Book-Ratings.csv               

"User-ID";"ISBN";"Book-Rating"
"276725";"034545104X";"0"
"276726";"0155061224";"5"
"276727";"0446520802";"0"
"276729";"052165615X";"3"
"276729";"0521795028";"6"
"276733";"2080674722";"0"
"276736";"3257224281";"8"
"276737";"0600570967";"6"
"276744";"038550120X";"7"
.....



BX-Books.csv

"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"
"0195153448";"Classical Mythology";"Mark P. O. Morford";"2002";"Oxford University Press";"http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg"
"0002005018";"Clara Callan";"Richard Bruce Wright";"2001";"HarperFlamingo Canada";"http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg"
"0060973129";"Decision in Normandy";"Carlo D'Este";"1991";"HarperPerennial";"http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg"
"0374157065";"Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It";"Gina Bari Kolata";"1999";"Farrar Straus Giroux";"http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg"
"0393045218";"The Mummies of Urumchi";"E. J. W. Barber";"1999";"W. W. Norton &amp; Company";"http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg"
```

In [7]:
print(df_ratings.head(1))
print("\n-----------------------------------------------\n")
print(df_books.head(1))

     user        isbn  rating
0  276725  034545104X     0.0

-----------------------------------------------

         isbn                title              author
0  0195153448  Classical Mythology  Mark P. O. Morford


### Prepare clean data



```
users - set
users_count_ratings - dict - user:ratings


books - set -from df_books - len 271379 
books2 -set - from df_ratings - len 340556
books_count_ratings - dict - book:ratings

authors - set 
authors_labels - list of authors when we sing indicates to dataset

```

#### users and books

Here I create set of unique users numbers.


Next is dict where user number show how many books user rating.


If I should delete users with less than 200 ratings.



In [8]:
#To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
users=set()
users={u for u in df_ratings['user']}
#print(users)

users_count_ratings={}
for u in users:
  users_count_ratings[u]=0

for u in df_ratings['user']:
  users_count_ratings[u]+=1


In [9]:
print(sorted(users_count_ratings.items(), key=lambda x: x[1], reverse=False))
print(sorted(users_count_ratings.items(), key=lambda x: x[1], reverse=True))

[(262145, 1), (2, 1), (262149, 1), (7, 1), (12, 1), (262158, 1), (19, 1), (20, 1), (262165, 1), (23, 1), (262168, 1), (262170, 1), (262174, 1), (262177, 1), (262179, 1), (36, 1), (262182, 1), (262183, 1), (42, 1), (262184, 1), (262191, 1), (51, 1), (262195, 1), (262196, 1), (262200, 1), (64, 1), (68, 1), (70, 1), (73, 1), (262218, 1), (77, 1), (78, 1), (79, 1), (81, 1), (82, 1), (83, 1), (86, 1), (88, 1), (262231, 1), (262232, 1), (91, 1), (95, 1), (97, 1), (100, 1), (102, 1), (107, 1), (109, 1), (110, 1), (262257, 1), (262258, 1), (262261, 1), (262267, 1), (125, 1), (262272, 1), (129, 1), (132, 1), (137, 1), (262280, 1), (141, 1), (144, 1), (151, 1), (262295, 1), (262301, 1), (262302, 1), (262303, 1), (262305, 1), (162, 1), (262312, 1), (169, 1), (262315, 1), (262316, 1), (262321, 1), (190, 1), (193, 1), (262338, 1), (202, 1), (207, 1), (262354, 1), (262355, 1), (212, 1), (262358, 1), (215, 1), (217, 1), (262364, 1), (221, 1), (224, 1), (226, 1), (230, 1), (262378, 1), (236, 1), (2623

In [10]:
#90,000 users. 
print("There is ",len(users), " users.")

max_value = max(users_count_ratings.values()) 
print("Max value of user count rating is: ", max_value)

There is  105283  users.
Max value of user count rating is:  13602


I create book set. 

I create dict .......................... 


In [11]:
books=set()
books={u for u in df_books['isbn']}

books2=set()
books2={u for u in df_ratings['isbn']}

print("books len", len(books),"\nbooks2 ", len(books2))
# there is something not right
# more books is in df_ratings  than in df_books ................ ????????

#books_count_ratings={}
#for b in books:
#  books_count_ratings[b]=0

#for index in df_ratings['isbn']:
#  books_count_ratings[index]+=1


books_count_ratings={}
for b in books2:
  books_count_ratings[b]=0

for index in df_ratings['isbn']:
  books_count_ratings[index]+=1
      



books len 271379 
books2  340556


In [12]:
#270,000 books
print(len(books))

271379


In [164]:
'''
books_without_title=set()  # create set od books without title
for book in books2:
  if book not in df_books['isbn'].values:
    books_without_title.add(book)
# 30 minutes  or much more .... to do this
print("There're ", len(books_without_title), "books without title.\nHere there are:  ", books_without_title)    
''' 

KeyboardInterrupt: ignored

## Create new dataset

### `data = []`

['user', 'isbn', 'rating']

In df_rating are some books which are not in df_books, so they dont have title.

So now we check row by row in data (list of ['user', 'isbn', 'rating']) if there is book without title (not in df_books).


In [167]:
 #NEVER grow a DataFrame!
#Accumulate data in a list, not a DataFrame.

# remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
data = []  # ['user', 'isbn', 'rating']
for row in df_ratings.values:
  if books_count_ratings[row[1]]>=100 and users_count_ratings[row[0]]>=200:
    #if row[1] not in books_without_title:
    if row[1] in df_books['isbn'].values:
      data.append(row)

# 5 min to do this 

print(len(data))

49517


In [168]:
df_ratings_new = pd.DataFrame(data,
             columns=['user', 'isbn', 'rating'])

In [169]:
df_ratings_new.head()

,user,isbn,rating
0,277427,002542730X,10.0
1,277427,0060930535,0.0
2,277427,0060934417,0.0
3,277427,0061009059,9.0
4,277427,0140067477,0.0


In [170]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


### merge df_ratings_new,  title => df (NOT NEEDED)

In [171]:
df= pd.merge(df_ratings_new, df_books, on = 'isbn')

In [172]:
df.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [120]:
df


,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
...,...,...,...,...,...
49512,238864,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...,Patricia Cornwell
49513,251843,0399149325,1.0,Portrait of a Killer: Jack the Ripper -- Case ...,Patricia Cornwell
49514,253821,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...,Patricia Cornwell
49515,265115,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...,Patricia Cornwell


### pivot matrix

In [180]:
pivotMatrix = df_ratings_new. pivot_table( 
    index = 'isbn', columns = 'user', values = 'rating').fillna(0)
print(pivotMatrix .head())
print(pivotMatrix .tail())

user        254     2276    2766    2977    ...  277427  277478  277639  278418
isbn                                        ...                                
002542730X     0.0     0.0     0.0     0.0  ...    10.0     0.0     0.0     0.0
0060008032     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
0060096195     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
006016848X     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
0060173289     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 888 columns]
user        254     2276    2766    2977    ...  277427  277478  277639  278418
isbn                                        ...                                
1573227331     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1573229326     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1573229571     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1592400876     0

### csr matrix

In [181]:
from scipy.sparse import csr_matrix
csrMatrix = csr_matrix(pivotMatrix .values)

In [182]:
csrMatrix

<727x888 sparse matrix of type '<class 'numpy.float64'>'
	with 12548 stored elements in Compressed Sparse Row format>

## Model

In [183]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(metric='cosine', algorithm='brute')
nbrs.fit(csrMatrix)
#distances, indices = nbrs.kneighbors(X,Y)
#print(distances)

NearestNeighbors(algorithm='brute', metric='cosine')

#### Trying random choise (NO NEEDED)

In [184]:
query_index = np.random.choice(pivotMatrix .shape[0])
print(query_index)

658


In [185]:
pivotMatrix.index[query_index]

'0743237188'

In [186]:
df_books[df_books['isbn']==pivotMatrix .index[query_index]].values[0][1]

'Fall On Your Knees (Oprah #45)'

In [ ]:
t = pivotMatrix .index=='0060096195'
print(t)

In [188]:
[i for i, x in enumerate(pivotMatrix .index=='0060096195') if x][0]

2

In [189]:
pivotMatrix .shape[0] # isbn 727  # title 673

727

In [193]:
distances, indices = nbrs.kneighbors(pivotMatrix .iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [194]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivotMatrix.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivotMatrix.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 0743237188:

1: 0375702709, with distance of 0.6639547744017598:
2: 0452282829, with distance of 0.6684035866182199:
3: 0375727345, with distance of 0.6821210032592122:
4: 0385505833, with distance of 0.7295557503021701:
5: 0425162443, with distance of 0.7315538293384463:


In [195]:
title = "The Queen of the Damned (Vampire Chronicles (Paperback))"

In [196]:
try:
  print(df_books[df_books['title']==title].values[0][0])
except:
  print("Dont' have that title or have typo!")

0345351525


## GET RECOMMENDS

In [197]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  try:
    book_isbn = df_books[df_books['title']==book].values[0][0]
  except:
    print("Dont' have that title or have typo!")
    return None

  ind = [i for i, x in enumerate(pivotMatrix .index==book_isbn) if x][0]
   
  distances, indices = nbrs.kneighbors(pivotMatrix .iloc[ind,:].values.reshape(1, -1), n_neighbors = 6)
  
  recommended_books = []
  recommended_books.append(book)
  recommended_books.append([])

  for i in range(0, len(distances.flatten())):
    if i == 0:
      pass
    else:
      new_book_isbn = pivotMatrix .index[indices.flatten()[i]]
      new_book = df_books[df_books['isbn']==new_book_isbn].values[0][1]
      new_book_dist = round(distances.flatten()[i],2)

      recommended_books[1].insert(0,[new_book, new_book_dist])

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [198]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉


# Herpers

https://blog.consdata.tech/2018/08/07/algorytmy-rekomendacyjne-przyklad-implementacji-w-pythonie.html

https://www.youtube.com/watch?v=kccT0FVK6OY
https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/KNNRecommendation.ipynb